# Data Wrangling Exercises
---

### 1. Read the case, department, and source data into their own spark dataframes.

In [1]:
# import spark functions/environment
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# create spark environment
spark = SparkSession.builder.getOrCreate()

In [3]:
# read each csv into spark dataframe
case = spark.read.csv('case.csv', header=True, inferSchema=True)
dept = spark.read.csv('dept.csv', header=True, inferSchema=True)
source = spark.read.csv('source.csv', header=True, inferSchema=True)

### 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

In [4]:
# save source data as json
source.write.json("sources_json")

In [5]:
# save source data as csv
source.write.csv("sources_csv")

Folders were created with the specified names and inside the folders is a file called SUCCESS and a json/csv fie of the data with a garbage-like name.

### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [9]:
# view dtypes
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [12]:
# view top 3 rows
source.show(3)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
+---------+----------------+
only showing top 3 rows



In [10]:
# view dtypes
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [13]:
# view top 3 rows
dept.show(3)

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|                YES|
|          Brush|Solid Waste Manag...|           Solid Waste|                YES|
|Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
+---------------+--------------------+----------------------+-------------------+
only showing top 3 rows



In [11]:
# view dtypes
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [16]:
# view first record
case.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [17]:
# rename column
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

In [21]:
# convert datatypes to time
case = (
    case.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)
# make boolean
case = (
    case.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)

---
### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [23]:
# prep for sql query
case.createOrReplaceTempView('case')

# write sql query
# latest
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM case
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 1
''').show()

+-------------+
|days_past_due|
+-------------+
|         1857|
+-------------+



In [27]:
# oldest
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_opened_date) AS days_since_opened
FROM case
WHERE NOT case_closed
ORDER BY days_since_opened DESC
LIMIT 1
''').show()

+-----------------+
|days_since_opened|
+-----------------+
|             1873|
+-----------------+



### 2. How many Stray Animal cases are there?

In [28]:
# count only stray animal cases
case.filter(case.service_request_type == 'Stray Animal').count()

26760

### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [39]:
# show only field operations division cases
# exclude officer standby request types
# show count
case.filter(case.dept_division == 'Field Operations')\
.filter(case.service_request_type != 'Officer Standby')\
.count()

113902

### 4. Convert the council_district column to a string column.

In [31]:
# convert council_district to string
case = case.withColumn('council_district', format_string('%04d', col('council_district')))

### 5. Extract the year from the case_closed_date column.

In [32]:
# get year from case_closed_date
case.select('case_closed_date', year('case_closed_date')).show(3)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
+-------------------+----------------------+
only showing top 3 rows



### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [40]:
# create new column multiplying each day by 24 hours
# select both columns
# show first 5 rows
case.withColumn('num_hours_late', case.num_days_late * 24)\
.select('num_days_late', 'num_hours_late')\
.show(5)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



### 7. Join the case data with the source and department data.

In [42]:
# join dataframes
df = case.join(dept, 'dept_division', 'left').join(source, 'source_id', 'left')
df.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 case_due_date          | 2020-09-26 00:42:00                  
 case_late              | false                                
 num_days_late          | -998.5087616000001                   
 case_closed            | true                                 
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 0005                                 
 dept_name              | Animal Care Se

### 8. Are there any cases that do not have a request source?

In [44]:
# select where source_id has no value
# sum those values
# show total records without request source
df.select(df.source_id.isNull().cast('int').alias('is_null'))\
.agg(sum('is_null'))\
.show()

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



### 9. What are the top 10 service request types in terms of number of requests?

In [46]:
# group by service request type
# get count for each type
# sort descending
# show top 10
df.groupby('service_request_type')\
.count()\
.sort(col('count').desc())\
.show(10, truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



### 10. What are the top 10 service request types in terms of average days late?

In [47]:
# select only late cases
# group by service request type
# aggregate by avg days late and number of cases
# sort descending
# show top 10
df.where('case_late')\
.groupBy('service_request_type')\
.agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))\
.sort(desc('n_days_late'))\
.show(10, truncate=False)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



### 11. Does number of days late depend on department?

In [53]:
# show only late cases
# group by department
# aggregate by avg days late
# sort by lateness
# round for neatness
df.filter('case_late')\
.groupby('dept_name')\
.agg(mean('num_days_late').alias('avg_days_late'))\
.sort('avg_days_late')\
.withColumn('avg_days_late', round(col('avg_days_late'), 2))\
.show(truncate=False)

+-------------------------+-------------+
|dept_name                |avg_days_late|
+-------------------------+-------------+
|Metro Health             |6.49         |
|Solid Waste Management   |7.15         |
|Trans & Cap Improvements |10.66        |
|Parks and Recreation     |22.43        |
|Animal Care Services     |23.45        |
|Code Enforcement Services|48.09        |
|Development Services     |67.22        |
|Customer Service         |88.18        |
|null                     |210.89       |
+-------------------------+-------------+



The average number of days a case is late does vary by department.

### 12. How do number of days late depend on department and request type?

In [55]:
# show only late cases
# group by dept name and request type
# aggregate by avg lateness
# round avg days late
# sort descending by lateness
# show top 10 records
df.filter("case_late")\
.groupby("dept_name", "service_request_type")\
.agg(avg("num_days_late").alias("days_late"))\
.withColumn("days_late", round(col("days_late"), 2))\
.sort(desc("days_late"))\
.show(10, truncate=False)

+-------------------------+--------------------------------------+---------+
|dept_name                |service_request_type                  |days_late|
+-------------------------+--------------------------------------+---------+
|null                     |Zoning: Recycle Yard                  |210.89   |
|Code Enforcement Services|Zoning: Junk Yards                    |200.21   |
|Code Enforcement Services|Structure/Housing Maintenance         |190.21   |
|Code Enforcement Services|Donation Container Enforcement        |171.09   |
|Code Enforcement Services|Storage of Used Mattress              |163.97   |
|Code Enforcement Services|Labeling for Used Mattress            |162.43   |
|Code Enforcement Services|Record Keeping of Used Mattresses     |154.0    |
|Code Enforcement Services|Signage Requied for Sale of Used Mattr|151.64   |
|Trans & Cap Improvements |Traffic Signal Graffiti               |137.65   |
|Code Enforcement Services|License Requied Used Mattress Sales   |128.8    |